# Preparing the Data

In [2]:
# Set your working directory
from os import getcwd
import pandas as pd
import numpy as np
import pickle as pk
from nltk import stem
import json
import matplotlib.pyplot as plt
import os
import json

fpath = getcwd()
print (fpath)

/Users/pierlim/PycharmProjects/sent_mining_CA


In [3]:
datapath = fpath + '/data/'
yjson = json.load(open(datapath + "yelp_2.json", "rb"))  # there are 35 reviews in this json
reviews = {}
i=0
for key0, val0 in yjson.items():  # keys are 0, 20  etc..
    for val in val0["review"]:
        reviews[i]= {"rating" : val["reviewRating"]["ratingValue"],
                     "review" : val["description"]}
        i += 1
print ("Total no of reviews " + str(len(reviews)))

Total no of reviews 35


## Collate Scraped Yelp Reviews - Train

In [4]:
# Read train data (6 restaurants out of 10 restaurants for each category) and collate 
total_reviews_train = {}
directory = './data/train/'

for filename in os.listdir(directory):
    
    if filename.endswith(".json"):
        print(filename)
        yjson = json.load(open(directory + filename, "rb")) 
        reviews = {}
        i=0
        for key0, val0 in yjson.items():  # keys are 0, 20  etc..
            for val in val0["review"]:
                reviews[i]= {"rating" : val["reviewRating"]["ratingValue"],
                             "review" : val["description"]}
                i += 1
        if bool(total_reviews_train)==False:
            total_reviews_train = dict(reviews)
        else:
            # need to re-enumerate the numbers here, else it will overwrite the total_reviews by key!
            reviews_to_update = {}
            cnt = len(total_reviews_train)
            for k,v in reviews.items():
                reviews_to_update[str(cnt)] = v
                cnt = cnt + 1
            total_reviews_train.update(reviews_to_update)
            #print(total_reviews)
            
print("Total train data = " + str(len(total_reviews_train)))
# with open('train_scraped.json', 'w') as outfile:
#     json.dump(total_reviews_train, outfile)

yelp_chinese_3_Wong Kei Restaurant.json
yelp_japanese_0_Bone Daddies.json
yelp_chinese_2_Hakkasan.json
yelp_chinese_0_Yauatcha.json
yelp_indian_4_Dishoom.json
yelp_chinese_9_Four Seasons.json
yelp_chinese_5_Ping Pong.json
yelp_french_8_Clos Maggiore.json
yelp_chinese_7_Ping Pong Soho.json
yelp_french_4_Le Mercury.json
yelp_indian_2_Punjab.json
yelp_japanese_1_Shoryu Ramen.json
yelp_indian_9_The Cinnamon Club.json
yelp_indian_6_Roti Chai.json
yelp_japanese_3_Nobu.json
yelp_japanese_4_Tokyo Diner.json
yelp_french_1_Savoir Faire.json
yelp_japanese_9_Nobu Berkeley St.json
yelp_indian_8_Veeraswamy.json
yelp_french_5_Balthazar.json
yelp_french_2_Le Relais de Venise L’Entrecôte.json
yelp_japanese_7_Kanada-Ya.json
yelp_indian_0_Dishoom.json
yelp_french_7_Chiltern Firehouse.json
Total train data = 5745


## Collate Scraped Yelp Reviews - Test (do by category) 

In [11]:
# Read train data (6 restaurants out of 10 restaurants for each category) and collate 
def create_dict_from_folder(directory): 
    total_reviews_test = {}
    #directory = './data/test/'

    for filename in os.listdir(directory):

        if filename.endswith(".json"):
            print(filename)
            yjson = json.load(open(directory + filename, "rb")) 
            reviews = {}
            i=0
            for key0, val0 in yjson.items():  # keys are 0, 20  etc..
                
                for val in val0["review"]:
                    reviews[i]= {"rating" : val["reviewRating"]["ratingValue"],
                                 "review" : val["description"],
                                 "id" : val0["id"],
                                 "name" : val0["name"],
                                 "category" : val0["category"]
                                }
                    i += 1
            if bool(total_reviews_test)==False:
                total_reviews_test = dict(reviews)
            else:
                # need to re-enumerate the numbers here, else it will overwrite the total_reviews by key!
                reviews_to_update = {}
                cnt = len(total_reviews_test)
                for k,v in reviews.items():
                    reviews_to_update[str(cnt)] = v
                    cnt = cnt + 1
                total_reviews_test.update(reviews_to_update)
                #print(total_reviews)

    print("Total test data = " + str(len(total_reviews_test)))
    # with open('test_scraped.json', 'w') as outfile:
    #     json.dump(total_reviews_test, outfile)
    return total_reviews_test
    
chinese = create_dict_from_folder('./data/test/chinese/')    
indian = create_dict_from_folder('./data/test/indian/')  
japanese = create_dict_from_folder('./data/test/japanese/')  
french = create_dict_from_folder('./data/test/french/')  
# df_chinese = create_df_from_folder('./data/test/chinese/')
# df_chinese

yelp_chinese_1_Lanzhou Noodle Bar.json
yelp_chinese_8_Golden Dragon.json
yelp_chinese_6_Joy King Lau.json
yelp_chinese_4_Dumplings’ Legend.json
Total test data = 650
yelp_indian_5_Dishoom.json
yelp_indian_3_Dishoom.json
yelp_indian_1_Tayyabs.json
yelp_indian_7_Masala Zone Soho.json
Total test data = 1369
yelp_japanese_5_Zuma.json
yelp_japanese_2_Misato.json
yelp_japanese_8_Sticks’n’Sushi.json
yelp_japanese_6_Roka.json
Total test data = 552
yelp_french_0_Sketch.json
yelp_french_6_L’Atelier de Joël Robuchon.json
yelp_french_9_Bar Boulud.json
yelp_french_3_Restaurant Gordon Ramsay.json
Total test data = 980


## Create the class labels -1 (neg), 1 (pos)
Note that we should not use lecturer-supplied train_neg.csv and train_pos.csv as the criteria for -ve and +ve may be different. 

We need to build our own training set based on what we scraped as well as the yelp_2.json

Use the ratings 1-3 as negative and 4-5 as positive in the Yelp reviews.

In [14]:
def mapPosNegR(score):
    rate = 1
    if score <= 3: rate = -1   # the result is dependent on the map fn
    return rate

actuals = []
for key, val in total_reviews_train.items():  # going through each review
    actuals.append(val['rating'])
    
actuals = list(map(mapPosNegR, actuals))  # actuals in terms of +1 or -1


In [15]:
df_reviews_train = pd.DataFrame(total_reviews_train).transpose()
df_reviews_train = df_reviews_train.reset_index().drop('index', axis=1)

In [16]:
df_actuals = pd.DataFrame(actuals)


In [17]:
df_reviews_train = pd.concat([df_reviews_train, pd.DataFrame(actuals)], axis=1)
#df_reviews.columns = {'rating', 'review', 'sentiment'}
#df_reviews_train

In [18]:
df_reviews_train.rename(columns={0:'sentiment'}, 
                 inplace=True)
df_reviews_train.head()

,rating,review,sentiment
0,5,In my younger days when lunch choices consiste...,1
1,3,"After going through yelp and tripadvisor, I wa...",-1
2,5,Ordered Sichuan Prawns and Singapore Rice Nood...,1
3,3,Wong Kei is one of the many options you'll fin...,-1
4,1,One of the worst experience in a restaurant in...,-1


Most reviews here are positive according to our rating criteria: it is an imbalanced set. 

In [19]:
df_reviews_train.groupby('sentiment').sum()

,rating,review
sentiment,,
-1,3166,"After going through yelp and tripadvisor, I wa..."
1,20322,In my younger days when lunch choices consiste...


In [20]:
df_reviews_train.tail()


,rating,review,sentiment
5740,1,"If I have to be honest, I have traveled extens...",-1
5741,2,This place is the worst. The crowd is old and...,-1
5742,5,Had an amazing dinner saturday night at Chilte...,1
5743,5,Excellent food. Great service but rushed a lit...,1
5744,4,I loved the decor and food but no where in all...,1


In [23]:
# now test
def process_to_df(dict_in):
    actuals = []
    for key, val in dict_in.items():  # going through each review
        actuals.append(val['rating'])

    actuals = list(map(mapPosNegR, actuals))  # actuals in terms of +1 or -1
    df_reviews_test = pd.DataFrame(dict_in).transpose()
    df_reviews_test = df_reviews_test.reset_index().drop('index', axis=1)
    df_reviews_test = pd.concat([df_reviews_test, pd.DataFrame(actuals)], axis=1)
    df_reviews_test.rename(columns={0:'sentiment'}, 
                     inplace=True)
    return df_reviews_test

df_chinese = process_to_df(chinese)
df_japanese = process_to_df(japanese)
df_indian = process_to_df(indian)
df_french = process_to_df(french)

In [24]:
df_chinese

,category,id,name,rating,review,sentiment
0,chinese,VZqEzifhvtYAxkWw6AuiCg,Lanzhou Noodle Bar,5,Let's be real here...London...and noodles?? Ye...,1
1,chinese,VZqEzifhvtYAxkWw6AuiCg,Lanzhou Noodle Bar,5,Who goes to London and has Chinese food? Me of...,1
2,chinese,VZqEzifhvtYAxkWw6AuiCg,Lanzhou Noodle Bar,5,My family and I visited London for a few days ...,1
3,chinese,VZqEzifhvtYAxkWw6AuiCg,Lanzhou Noodle Bar,5,This place is so unassuming and very easy to p...,1
4,chinese,VZqEzifhvtYAxkWw6AuiCg,Lanzhou Noodle Bar,3,It's a small and cramped little hole in the wa...,-1
5,chinese,VZqEzifhvtYAxkWw6AuiCg,Lanzhou Noodle Bar,2,"Don't get me wrong, I love noodles. I love all...",-1
6,chinese,VZqEzifhvtYAxkWw6AuiCg,Lanzhou Noodle Bar,2,Worst meal of my trip.\nWe ordered two chicken...,-1
7,chinese,VZqEzifhvtYAxkWw6AuiCg,Lanzhou Noodle Bar,4,Tiny place with great noodles! We stopped here...,1
8,chinese,VZqEzifhvtYAxkWw6AuiCg,Lanzhou Noodle Bar,4,A good place for a reasonably priced meal in t...,1
9,chinese,VZqEzifhvtYAxkWw6AuiCg,Lanzhou Noodle Bar,5,Went at about 6:30 on a busy Saturday night. ...,1


In [168]:
df_reviews_test.groupby('sentiment').sum()

,rating,review
sentiment,,
-1,2019,It's a small and cramped little hole in the wa...
1,12370,Let's be real here...London...and noodles?? Ye...


In [25]:

df_reviews_train.to_csv('./data/df_reviews_train.csv', index=False)
#df_reviews_test.to_csv('./data/df_reviews_test.csv', index=False)

df_chinese.to_csv('./data/df_test_chinese.csv', index=False)
df_japanese.to_csv('./data/df_test_japanese.csv', index=False)
df_indian.to_csv('./data/df_test_indian.csv', index=False)
df_french.to_csv('./data/df_test_french.csv', index=False)